In [1]:
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import pandas as pd

In [2]:
pd.options.display.max_colwidth = 3000
pd.options.display.max_rows = 3000

In [3]:
data = pd.read_csv('./master_data.csv')
data.head(2)

,ADR,text,category,type,date,org/aug
0,ADR-2022-100004022857,"# Lee, Terrilynn, Marie 000029128 I=0000 29128 1 AV 0.423 TERRILYNN MARIE LEE 524 S ACADEMY ST MEDINA NY 14103 This letter is a summary of benefits you currently receive from the Department of Veterans Affairs (VA). We are providing this letter to disabled Veterans to use in applying for benefits such as housing entitlements, free or reduced state park annual memberships, state or local property or vehicle tax relief, civil service preference, or any other program or entitlement in which verification of VA benefits is required. Please safeguard this important document. This letter replaces VA Form 20-5455, and is considered an official record of your VA entitlement. -- America is Grateful to You for Your Service -- Our records contain the following information: Personal Claim Information: Your VA claim number is: 103 66 6110 You are the Veteran Military Information: Your character(s) of discharge and service date(s) include: Air Force, Honorable, 25-Jun-1986 - 25-Jun-1995 (You may have additional periods of service not listed above) VA Benefits Information: Service-connected disability: Yes Your combined service-connected evaluation is: 100 PERCENT The effective date of the last change to your current award was: 01-DEC-2020 Your current monthly award amount is: $3,146.42 0.570 oz 596133-002-0/3946703 0029128 0116509 |=000000 Are you considered to be totally and permanently disabled due to your service-connected disabilities: Yes You should contact your state or local office of Veterans' affairs for information on any tax, license, or fee-related benefits for which you may be eligible. State offices of Veterans' affairs are available at http://www.va.gov/statedva.htm. Need Additional Information or Verification? If you have any questions about this letter or need additional verification of VA benefits, please call us at 1-800-827-1000. If you use a Telecommunications Device for the Deaf (TDD), the federal relay number is 711. Send electronic inquiries through the Internet at https://iris.custhelp.va.gov/. Sincerely yours, Regional Office Director Enclosure(s): What Things Affect Your Rights To Payment NE201221 MAIL WWCZX50 AS OF $50+ SET R FOR DETAILS",Award Letter(s),LD,20220711135206,org
1,ADR-2022-100004025366,"BeBenefits A Service of the Department of Veterans Affairs and the Dashboard v My Gateway to Benefit Information Department of Defense Apply Manage Learn National Resource Directory Employment Center Contact Q Search For feature and outage events, refer to the Outage Calendar link at the bottom of the page. To ensure your security and privacy, when using a shared or public computer, make sure you always log out of VA/DoD eBenefits and then close all browser windows when finished accessing the site. Dashboard Disabilities My Profile Total Combined Disability Add Disabilities Personal Information You have a 90% final degree of 90% Submit a claim to make any Representative disability. This percentage determines the amount of benefit updates to the disabilities that you Disabilities pay you will receive. believe are related to your military service. Dependents How is this calculated Apply Now. My Documents My Claims & Appeals Work in Process Rated Disabilities Historical Claims Historical Appeals Disability Rating Decision Related To Effective Date My Benefits & Payments posttraumatic stress disorder 70% Service Connected PTSD - Personal 01/27/2021 (PTSD), major depressive disorder, Benefits and alcohol use disorder (also Trauma claimed as sleep disturbances) Payment History eating disorders Not Service Connected Messages migraine 50% Service Connected 10/19/2021 Account Activity Pending Disabilities You have no Pending Disabilities to display at this time.",Award Letter(s),LD,20220711135206,org


In [ ]:
ocrAug= nac.OcrAug() 

'''bertWordEmbsAug= naw.ContextualWordEmbsAug(model_path='distilroberta-base', 
                                model_type='bert',
                                action="insert", 
                                aug_p=0.8,
                                device='cuda')'''

spellingAug = naw.SpellingAug(aug_p=0.3)

randomSentAug = nas.random.RandomSentAug(mode='neighbor', action='swap')

In [ ]:
def ocrAugument(text):
    return ocrAug.augment(text, num_thread=6)

# def wordEmbsAugument(text):
#     return bertWordEmbsAug.augment(text)

def swapSentAugument(text):
    return randomSentAug.augment(text, num_thread=6)

def wordSpellingAug(txt):
    return spellingAug.augment(txt, num_thread=6)

In [ ]:
ocrAug = ocrAugument(list(data['text'].values))
# ocrAug = ocrAugument(list(data['text'][data['new_category']=='others'].values))
len(ocrAug)

In [ ]:
spellAug = wordSpellingAug(list(data['text'].values))
len(spellAug)

In [ ]:
swapSentAug = swapSentAugument(list(data['text'].values))
len(swapSentAug)

In [ ]:
def addAugText(df, augText):
    tempDf = pd.DataFrame(columns=['ADR', 'text', 'type', 'date', 'org/aug'])
    tempDf['ADR'] = df['ADR']
    tempDf['text'] = np.squeeze(augText) 
    tempDf['category'] = df['category']
    tempDf['type'] = df['type']
    tempDf['date'] = df['date']
    tempDf['org/aug'] = "aug"
    return tempDf

In [ ]:
ocrAugDf = addAugText(data, ocrAug)
#wordEmbsAugDf = addAugText(data, wordEmbsAug)
spellAugDf = addAugText(data, spellAug)
swapSentAugDf = addAugText(data, swapSentAug)

In [ ]:
dataAug = pd.concat([data,ocrAugDf,spellAugDf,swapSentAugDf])
dataAug.shape

In [ ]:
dataAug.to_csv('./master_data_AUG.csv', header=True, index=False)